# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import math
import requests
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'aapl'
api_url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()[0]
print(data)

{'beta': 1.2846989112885705, 'totalCash': 55872000000, 'currentDebt': 120075000000, 'revenue': 385095000000, 'grossProfit': 166288000000, 'totalRevenue': 385095000000, 'EBITDA': 123788000000, 'revenuePerShare': 24.48, 'revenuePerEmployee': 2619693.88, 'debtToEquity': 5.343801280607484, 'profitMargin': 0.2449291733208689, 'enterpriseValue': 3082776431160, 'enterpriseValueToRevenue': 8.01, 'priceToSales': 7.87, 'priceToBook': 48.73, 'forwardPERatio': 32.28186215252416, 'pegRatio': -7.363505377785372, 'peHigh': 13.377022411661969, 'peLow': 10.016056606658356, 'week52highDate': '2023-07-19', 'week52lowDate': '2023-01-03', 'putCallRatio': 0.5013102582300595, 'companyName': 'Apple Inc', 'marketcap': 3029033431160, 'week52high': 198.23, 'week52low': 123.64, 'week52highSplitAdjustOnly': 198.23, 'week52highDateSplitAdjustOnly': '2023-07-19', 'week52lowSplitAdjustOnly': 124.17, 'week52lowDateSplitAdjustOnly': '2023-01-03', 'week52change': 0.16922139132367064, 'sharesOutstanding': 15728702000, 'f

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']


0.21394122426646955

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])


A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    batch_api_call_url_as = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    advanced_stats = requests.get(batch_api_call_url).json()
    quote = requests.get(batch_api_call_url_as).json()
    for i in range(0,len(quote)):
        final_dataframe = final_dataframe._append(
            pd.Series(
                [
                    quote[i]['symbol'],
                    quote[i]['latestPrice'],
                    advanced_stats[i]['year1ChangePercent'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )
final_dataframe


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,125.05,-0.020600,N/A
1,AAL,15.98,0.134424,N/A
2,AAP,72.03,-0.630059,N/A
3,AAPL,191.17,0.213941,N/A
4,ABBV,149.05,0.107203,N/A
...,...,...,...,...
500,YUM,135.62,0.121098,N/A
501,ZBH,125.79,0.134245,N/A
502,ZBRA,249.55,-0.254145,N/A
503,ZION,37.41,-0.281799,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,79.92,2.281287,N/A
1,390,RCL,104.72,1.869613,N/A
2,345,NVDA,445.15,1.391784,N/A
3,194,FTI,18.25,1.181818,N/A
4,370,PHM,81.45,1.028766,N/A
5,332,NFLX,431.00,0.940656,N/A
6,198,GE,112.36,0.920722,N/A
7,84,CCL,17.56,0.895187,N/A
8,50,AVGO,884.98,0.738642,N/A
9,132,DHI,123.15,0.713892,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the portfolo size')
    try:
        float(portfolio_size)
    except ValueError:
        print('Please give a integer value\n')
        portfolio_size = input('Enter the portfolo size')

portfolio_input()
print(portfolio_size)


        
        

Enter the portfolo size 1000000000


1000000000


In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])

final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,79.92,2.281287,250250
1,390,RCL,104.72,1.869613,190985
2,345,NVDA,445.15,1.391784,44928
3,194,FTI,18.25,1.181818,1095890
4,370,PHM,81.45,1.028766,245549
5,332,NFLX,431.00,0.940656,46403
6,198,GE,112.36,0.920722,177999
7,84,CCL,17.56,0.895187,1138952
8,50,AVGO,884.98,0.738642,22599
9,132,DHI,123.15,0.713892,162403


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [69]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]


hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    batch_api_call_url_as = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    advanced_stats = requests.get(batch_api_call_url).json()
    quote = requests.get(batch_api_call_url_as).json()
    for i in range(0,len(quote)):
        hqm_dataframe = hqm_dataframe._append(
            pd.Series(
                [
                    quote[i]['symbol'],
                    quote[i]['latestPrice'],
                    'N/A',
                    advanced_stats[i]['year1ChangePercent'],
                    'N/A',
                    advanced_stats[i]['month6ChangePercent'],
                    'N/A',
                    advanced_stats[i]['month3ChangePercent'],
                    'N/A',
                    advanced_stats[i]['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index = hqm_columns
            ),
            ignore_index = True
        )
hqm_dataframe.dropna(inplace=True)
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.05,N/A,-0.020600,N/A,-0.175805,N/A,-0.048811,N/A,0.062037,N/A,N/A
1,AAL,15.98,N/A,0.134424,N/A,-0.065064,N/A,0.158315,N/A,-0.110925,N/A,N/A
2,AAP,72.03,N/A,-0.630059,N/A,-0.544179,N/A,-0.436940,N/A,-0.005876,N/A,N/A
3,AAPL,191.17,N/A,0.213941,N/A,0.282377,N/A,0.145800,N/A,-0.007166,N/A,N/A
4,ABBV,149.05,N/A,0.107203,N/A,0.052371,N/A,-0.003851,N/A,0.121028,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.62,N/A,0.121098,N/A,0.057874,N/A,-0.050875,N/A,-0.025623,N/A,N/A
501,ZBH,125.79,N/A,0.134245,N/A,0.011215,N/A,-0.100808,N/A,-0.118269,N/A,N/A
502,ZBRA,249.55,N/A,-0.254145,N/A,-0.300406,N/A,-0.055943,N/A,-0.184836,N/A,N/A
503,ZION,37.41,N/A,-0.281799,N/A,-0.308062,N/A,0.558246,N/A,0.363738,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [70]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
        for time_period in time_periods:
            change_col = f'{time_period} Price Return'
            percentile_col = f'{time_period} Return Percentile'
            hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col],hqm_dataframe.loc[row, change_col])/100

hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.05,N/A,-0.020600,0.327198,-0.175805,0.09816,-0.048811,0.165644,0.062037,0.734151,N/A
1,AAL,15.98,N/A,0.134424,0.621677,-0.065064,0.306748,0.158315,0.754601,-0.110925,0.04499,N/A
2,AAP,72.03,N/A,-0.630059,0.00409,-0.544179,0.00409,-0.436940,0.00409,-0.005876,0.331288,N/A
3,AAPL,191.17,N/A,0.213941,0.750511,0.282377,0.96319,0.145800,0.715746,-0.007166,0.321063,N/A
4,ABBV,149.05,N/A,0.107203,0.556237,0.052371,0.631902,-0.003851,0.292434,0.121028,0.91002,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.62,N/A,0.121098,0.591002,0.057874,0.662577,-0.050875,0.161554,-0.025623,0.212679,N/A
501,ZBH,125.79,N/A,0.134245,0.615542,0.011215,0.529652,-0.100808,0.059305,-0.118269,0.038855,N/A
502,ZBRA,249.55,N/A,-0.254145,0.05726,-0.300406,0.02863,-0.055943,0.147239,-0.184836,0.00818,N/A
503,ZION,37.41,N/A,-0.281799,0.04499,-0.308062,0.02454,0.558246,0.99182,0.363738,1.0,N/A


### Calculatingd the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [71]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
         momentum_percentile.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percentile)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,125.05,N/A,-0.020600,0.327198,-0.175805,0.09816,-0.048811,0.165644,0.062037,0.734151,0.331288
1,AAL,15.98,N/A,0.134424,0.621677,-0.065064,0.306748,0.158315,0.754601,-0.110925,0.04499,0.432004
2,AAP,72.03,N/A,-0.630059,0.00409,-0.544179,0.00409,-0.436940,0.00409,-0.005876,0.331288,0.08589
3,AAPL,191.17,N/A,0.213941,0.750511,0.282377,0.96319,0.145800,0.715746,-0.007166,0.321063,0.687628
4,ABBV,149.05,N/A,0.107203,0.556237,0.052371,0.631902,-0.003851,0.292434,0.121028,0.91002,0.597648
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.62,N/A,0.121098,0.591002,0.057874,0.662577,-0.050875,0.161554,-0.025623,0.212679,0.406953
501,ZBH,125.79,N/A,0.134245,0.615542,0.011215,0.529652,-0.100808,0.059305,-0.118269,0.038855,0.310838
502,ZBRA,249.55,N/A,-0.254145,0.05726,-0.300406,0.02863,-0.055943,0.147239,-0.184836,0.00818,0.060327
503,ZION,37.41,N/A,-0.281799,0.04499,-0.308062,0.02454,0.558246,0.99182,0.363738,1.0,0.515337


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [72]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
78,CARR,58.10,N/A,0.484393,0.94274,0.261934,0.95501,0.422164,0.98364,0.186482,0.97546,0.964213
79,CAT,281.45,N/A,0.601559,0.973415,0.186455,0.899796,0.343221,0.96728,0.174534,0.96728,0.951943
194,FTI,18.25,N/A,1.181818,0.993865,0.383551,0.98773,0.400778,0.97955,0.083032,0.828221,0.947342
37,ANET,177.76,N/A,0.513403,0.957055,0.328537,0.973415,0.313380,0.948875,0.093916,0.862986,0.935583
166,ETN,217.10,N/A,0.526255,0.961145,0.365021,0.981595,0.269802,0.92638,0.088364,0.854806,0.930982
370,PHM,81.45,N/A,1.028766,0.99182,0.393417,0.989775,0.273243,0.928425,0.075695,0.795501,0.92638
275,LB,79.92,N/A,2.281287,1.0,0.815831,0.997955,0.215421,0.879346,0.079271,0.815951,0.923313
107,COG,22.25,N/A,0.340483,0.877301,0.180873,0.893661,0.255629,0.91411,0.249298,0.989775,0.918712
345,NVDA,445.15,N/A,1.391784,0.99591,1.039969,1.0,0.569591,0.993865,0.046499,0.676892,0.916667
66,BKR,35.65,N/A,0.447182,0.92638,0.155367,0.860941,0.318004,0.95501,0.124960,0.92229,0.916155


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [73]:
portfolio_input()

Enter the portfolo size 100000


In [74]:
for row in hqm_dataframe.index:
    hqm_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[row,'Price'])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
78,CARR,58.10,344234,0.484393,0.94274,0.261934,0.95501,0.422164,0.98364,0.186482,0.97546,0.964213
79,CAT,281.45,71060,0.601559,0.973415,0.186455,0.899796,0.343221,0.96728,0.174534,0.96728,0.951943
194,FTI,18.25,1095890,1.181818,0.993865,0.383551,0.98773,0.400778,0.97955,0.083032,0.828221,0.947342
37,ANET,177.76,112511,0.513403,0.957055,0.328537,0.973415,0.313380,0.948875,0.093916,0.862986,0.935583
166,ETN,217.10,92123,0.526255,0.961145,0.365021,0.981595,0.269802,0.92638,0.088364,0.854806,0.930982
370,PHM,81.45,245549,1.028766,0.99182,0.393417,0.989775,0.273243,0.928425,0.075695,0.795501,0.92638
275,LB,79.92,250250,2.281287,1.0,0.815831,0.997955,0.215421,0.879346,0.079271,0.815951,0.923313
107,COG,22.25,898876,0.340483,0.877301,0.180873,0.893661,0.255629,0.91411,0.249298,0.989775,0.918712
345,NVDA,445.15,44928,1.391784,0.99591,1.039969,1.0,0.569591,0.993865,0.046499,0.676892,0.916667
66,BKR,35.65,561009,0.447182,0.92638,0.155367,0.860941,0.318004,0.95501,0.124960,0.92229,0.916155


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [76]:
writer = pd.ExcelWriter('momentum_strategy.xlsx',engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [77]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [78]:
column_formats = {
   'A' : ['Ticker', string_template],
   'B' : ['Price' , dollar_template],
   'C' : ['Number of Shares to Buy' , integer_template],
   'D' : ['One-Year Price Return', percent_template],
   'E' : ['One-Year Return Percentile', percent_template],
   'F' : ['Six-Month Price Return', percent_template],
   'G' : ['Six-Month Return Percentile', percent_template],
   'H' : ['Three-Month Price Return', percent_template],
   'I' : ['Three-Month Return Percentile', percent_template],
   'J' : ['One-Month Price Return', percent_template],
   'K' : ['One-Month Return Percentile', percent_template],
   'L' : ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column( f'{column}:{column}' , 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0],column_formats[column][1])



## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [79]:
writer.close()